<a href="https://colab.research.google.com/github/Gainward777/TalkingFace/blob/main/TFaceUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installs

!pip install gradio


!pip install elevenlabs -U


!pip install langchain
!pip install openai

!pip3 install python-dotenv


#!git clone https://github.com/yzhou359/MakeItTalk
!git clone https://github.com/Gainward777/MakeItTalk
%cd MakeItTalk/
!export PYTHONPATH=/content/MakeItTalk:$PYTHONPATH
!pip install -r requirements.txt
!pip install tensorboardX

!pip install Resemblyzer

!pip install librosa==0.9.1




In [ ]:
#@title Load Libs and Downloads

%cd MakeItTalk/
!mkdir examples/dump
!mkdir examples/ckpt
!pip install gdown
!gdown -O examples/ckpt/ckpt_autovc.pth https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
!gdown -O examples/ckpt/ckpt_content_branch.pth https://drive.google.com/uc?id=1r3bfEvTVl6pCNw5xwUhEglwDHjWtAqQp
!gdown -O examples/ckpt/ckpt_speaker_branch.pth https://drive.google.com/uc?id=1rV0jkyDqPW-aDJcj7xSO6Zt1zSXqn1mu
!gdown -O examples/ckpt/ckpt_116_i2i_comb.pth https://drive.google.com/uc?id=1i2LJXKp-yWKIEEgJ7C6cE3_2NirfY_0a
!gdown -O examples/dump/emb.pickle https://drive.google.com/uc?id=18-0CYl5E6ungS3H4rRSHjfYvvm-WwjTI

import sys
sys.path.append("thirdparty/AdaptiveWingLoss")
import os, glob
import numpy as np
import cv2
import argparse
from src.approaches.train_image_translation import Image_translation_block
import torch
import pickle
import face_alignment
from src.autovc.AutoVC_mel_Convertor_retrain_version import AutoVC_mel_Convertor
import shutil
import time
import util.utils as util
from scipy.signal import savgol_filter
from src.approaches.train_audio2landmark import Audio2landmark_model

from resemblyzer import preprocess_wav, VoiceEncoder
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resemblyzer_encoder = VoiceEncoder(device=device)

predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType.THREE_D, device='cpu', flip_input=True)



from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from dotenv import find_dotenv, load_dotenv
import requests

import openai
#openai.api_key=
#OPENAI_API_KEY =



from scipy.io.wavfile import write as write_wav #to temporarily save audio

from elevenlabs import generate, play, set_api_key

set_api_key('') #elevenlabs api key




from PIL import Image
import gradio as gr
import shutil


def get_spk_emb_prepr(audio_file_dir, resemblyzer_encoder, segment_len=960000):
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #resemblyzer_encoder = VoiceEncoder(device=device)

    wav = preprocess_wav(audio_file_dir)
    l = len(wav) // segment_len # segment_len = 16000 * 60
    l = np.max([1, l])
    all_embeds = []
    for i in range(l):
        mean_embeds, cont_embeds, wav_splits = resemblyzer_encoder.embed_utterance(
            wav[segment_len * i:segment_len* (i + 1)], return_partials=True, rate=2)
        all_embeds.append(mean_embeds)
    all_embeds = np.array(all_embeds)
    mean_embed = np.mean(all_embeds, axis=0)

    return mean_embed, all_embeds


load_dotenv(find_dotenv())

def get_response_form_ai(human_input, api_key, background, temperature, memory_len):
    template=f"{background}" + """
    {history}
    user: {human_input}
    agent:
    """

    prompt=PromptTemplate(input_variables={"history", "human_input"}, template=template)

    chatgpt_chain=LLMChain(llm=OpenAI(openai_api_key=api_key,temperature=temperature), prompt=prompt,
                           verbose=True, memory=ConversationBufferWindowMemory(k=memory_len))

    output=chatgpt_chain.predict(human_input=human_input)

    return output

/content/MakeItTalk
Downloading...
From: https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
To: /content/MakeItTalk/examples/ckpt/ckpt_autovc.pth
100% 172M/172M [00:02<00:00, 84.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1r3bfEvTVl6pCNw5xwUhEglwDHjWtAqQp
To: /content/MakeItTalk/examples/ckpt/ckpt_content_branch.pth
100% 7.88M/7.88M [00:00<00:00, 50.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rV0jkyDqPW-aDJcj7xSO6Zt1zSXqn1mu
To: /content/MakeItTalk/examples/ckpt/ckpt_speaker_branch.pth
100% 15.4M/15.4M [00:00<00:00, 60.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i2LJXKp-yWKIEEgJ7C6cE3_2NirfY_0a
To: /content/MakeItTalk/examples/ckpt/ckpt_116_i2i_comb.pth
100% 839M/839M [00:07<00:00, 116MB/s]
Downloading...
From: https://drive.google.com/uc?id=18-0CYl5E6ungS3H4rRSHjfYvvm-WwjTI
To: /content/MakeItTalk/examples/dump/emb.pickle
100% 30.9M/30.9M [00:00<00:00, 76.8MB/s]
Loaded the voice encoder model on cuda in 8.84 seconds.


Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth
100%|██████████| 85.7M/85.7M [00:05<00:00, 15.5MB/s]
Downloading: "https://www.adrianbulat.com/downloads/python-fan/3DFAN4-4a694010b9.zip" to /root/.cache/torch/hub/checkpoints/3DFAN4-4a694010b9.zip
100%|██████████| 91.9M/91.9M [00:06<00:00, 13.8MB/s]
Downloading: "https://www.adrianbulat.com/downloads/python-fan/depth-6c4283c0e0.zip" to /root/.cache/torch/hub/checkpoints/depth-6c4283c0e0.zip
100%|██████████| 224M/224M [00:20<00:00, 11.3MB/s]


In [1]:
#@title Constants
default_head_name=''
AMP_LIP_SHAPE_X = 2 #@param {type:"slider", min:0.5, max:5.0, step:0.1}

#@markdown Amplify the lip motion in vertical direction
AMP_LIP_SHAPE_Y = 2 #@param {type:"slider", min:0.5, max:5.0, step:0.1}

#@markdown Amplify the head pose motion (usually smaller than 1.0, put it to 0. for a static head pose)
AMP_HEAD_POSE_MOTION = 0.35 #@param {type:"slider", min:0.0, max:1.0, step:0.05}

#@markdown Add naive eye blink
ADD_NAIVE_EYE = True  #@param ["False", "True"] {type:"raw"}

#@markdown If your image has an opened mouth, put this as True, else False
CLOSE_INPUT_FACE_MOUTH = False  #@param ["False", "True"] {type:"raw"}


#@markdown # Landmark Adjustment

#@markdown Adjust upper lip thickness (postive value means thicker)
UPPER_LIP_ADJUST = 0 #@param {type:"slider", min:-3.0, max:3.0, step:1.0}

#@markdown Adjust lower lip thickness (postive value means thicker)
LOWER_LIP_ADJUST = 1 #@param {type:"slider", min:-3.0, max:3.0, step:1.0}

#@markdown Adjust static lip width (in multipication)
LIP_WIDTH_ADJUST = 1.01 #@param {type:"slider", min:0.8, max:1.2, step:0.01}

#@markdown Eye size (test func)
LARGER_EYE=2 #@param {type:"slider", min:0, max:4, step:0.1}

#@markdown # Voice Controllers
#from google.colab import files
'''
print('Upload voice example. It must be ".wav ')
voice_path = list(files.upload().keys())[0]
'''

TEMPERATURE=0.7 #@param {type:"slider", min:0.0, max:1.0, step:0.01}

BACKGROUND = " you are as a role of Batman - Gotham Knight. Now lets playing the following requirements:  1) your name is Bruce Wayne, but no one knows that you are Batman;  2) you are 29 years old, you thunderstorm of crime, but sometimes you just want to be a gardener;  3) you have language addiction, sometimes you say [laughter] or [clears throat] at the and of the santance;  4) little by little you go crazy, sometimes you think you're really a bat;  5) Don't be overly enthusiastsic, but be cringe, don't be overly negative, don't be too boring." #@param {type: "string"}

OPENAI_API_KEY = "" #@param {type: "string"}

ELEVEN_LABS_API_KEY = "" #@param {type: "string"}

MEMORY_LEN =2 #@param {type: "integer"}

In [ ]:
#@title _for test
!pip install gradio
import gradio as gr
import shutil

set_api_key(ELEVEN_LABS_API_KEY)

In [ ]:
#@title Set Funks
def preprocess():
    global opt_parser
    global shape_3d
    global scale
    global shift
    global AMP_LIP_SHAPE_X
    global AMP_LIP_SHAPE_Y
    global AMP_HEAD_POSE_MOTION
    global ADD_NAIVE_EYE
    global CLOSE_INPUT_FACE_MOUTH
    global UPPER_LIP_ADJUST
    global LOWER_LIP_ADJUST
    global LIP_WIDTH_ADJUST
    global LARGER_EYE
    global img


    parser = argparse.ArgumentParser()
    parser.add_argument('--jpg', type=str, default='{}.jpg'.format(default_head_name))
    parser.add_argument('--close_input_face_mouth', default=CLOSE_INPUT_FACE_MOUTH, action='store_true')

    parser.add_argument('--load_AUTOVC_name', type=str, default='examples/ckpt/ckpt_autovc.pth')
    parser.add_argument('--load_a2l_G_name', type=str, default='examples/ckpt/ckpt_speaker_branch.pth')
    parser.add_argument('--load_a2l_C_name', type=str, default='examples/ckpt/ckpt_content_branch.pth') #ckpt_audio2landmark_c.pth')
    parser.add_argument('--load_G_name', type=str, default='examples/ckpt/ckpt_116_i2i_comb.pth') #ckpt_image2image.pth') #ckpt_i2i_finetune_150.pth') #c

    parser.add_argument('--amp_lip_x', type=float, default=AMP_LIP_SHAPE_X)
    parser.add_argument('--amp_lip_y', type=float, default=AMP_LIP_SHAPE_Y)
    parser.add_argument('--amp_pos', type=float, default=AMP_HEAD_POSE_MOTION)
    parser.add_argument('--reuse_train_emb_list', type=str, nargs='+', default=[]) #  ['iWeklsXc0H8']) #['45hn7-LXDX8']) #['E_kmpT-EfOg']) #'iWeklsXc0H8', '29k8RtSUjE0', '45hn7-LXDX8',
    parser.add_argument('--add_audio_in', default=False, action='store_true')
    parser.add_argument('--comb_fan_awing', default=False, action='store_true')
    parser.add_argument('--output_folder', type=str, default='examples')

    parser.add_argument('--test_end2end', default=True, action='store_true')
    parser.add_argument('--dump_dir', type=str, default='', help='')
    parser.add_argument('--pos_dim', default=7, type=int)
    parser.add_argument('--use_prior_net', default=True, action='store_true')
    parser.add_argument('--transformer_d_model', default=32, type=int)
    parser.add_argument('--transformer_N', default=2, type=int)
    parser.add_argument('--transformer_heads', default=2, type=int)
    parser.add_argument('--spk_emb_enc_size', default=16, type=int)
    parser.add_argument('--init_content_encoder', type=str, default='')
    parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
    parser.add_argument('--reg_lr', type=float, default=1e-6, help='weight decay')
    parser.add_argument('--write', default=False, action='store_true')
    parser.add_argument('--segment_batch_size', type=int, default=1, help='batch size')
    parser.add_argument('--emb_coef', default=3.0, type=float)
    parser.add_argument('--lambda_laplacian_smooth_loss', default=1.0, type=float)
    parser.add_argument('--use_11spk_only', default=False, action='store_true')
    parser.add_argument('-f')

    opt_parser = parser.parse_args()

    %cd MakeItTalk

    img =cv2.imread('examples/' + opt_parser.jpg)
    #moved to imports
    #predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType.THREE_D, device='cpu', flip_input=True)
    shapes = predictor.get_landmarks(img)
    if (not shapes or len(shapes) != 1):
        print('Cannot detect face landmarks. Exit.')
        exit(-1)
    shape_3d = shapes[0]

    if(opt_parser.close_input_face_mouth):
        util.close_input_face_mouth(shape_3d)


    shape_3d[48:, 0] = (shape_3d[48:, 0] - np.mean(shape_3d[48:, 0])) * LIP_WIDTH_ADJUST + np.mean(shape_3d[48:, 0]) # wider lips
    shape_3d[49:54, 1] += UPPER_LIP_ADJUST           # thinner upper lip
    shape_3d[55:60, 1] -= LOWER_LIP_ADJUST          # thinner lower lip
    shape_3d[[37,38,43,44], 1] -=LARGER_EYE    # larger eyes
    shape_3d[[40,41,46,47], 1] +=LARGER_EYE    # larger eyes

    shape_3d, scale, shift = util.norm_input_face(shape_3d)



def generation(SAY_SOMETHING):
    global BACKGROUND
    global TEMPERATURE
    global MEMORY_LEN

    if os.path.exists('/content/MakeItTalk/examples/audio.wav'):
        os.remove('/content/MakeItTalk/examples/audio.wav')
    if os.path.exists("/content/MakeItTalk/examples/my_new_img_pred_fls_audio_audio_embed.mp4"):
        os.remove("/content/MakeItTalk/examples/my_new_img_pred_fls_audio_audio_embed.mp4")
    if os.path.exists('/content/MakeItTalk/examples/tmp.wav'):
        os.remove('/content/MakeItTalk/examples/tmp.wav')

    openai.api_key=OPENAI_API_KEY
    text_out=get_response_form_ai(SAY_SOMETHING,
                                  openai.api_key,
                                  BACKGROUND,
                                  TEMPERATURE,
                                  MEMORY_LEN)

    audio = generate(
    text=text_out,
    voice="Arnold"
    )

    with open('/content/MakeItTalk/examples/audio.wav', mode='bx') as f:
        f.write(audio)

    if os.path.exists('/content/MakeItTalk/examples/M6_04_16k.wav'):
        os.remove('/content/MakeItTalk/examples/M6_04_16k.wav')


    #%cd MakeItTalk

    au_data = []
    au_emb = []
    ains = glob.glob1('examples', '*.wav')
    ains = [item for item in ains if item is not 'tmp.wav']
    ains.sort()
    for ain in ains:
        os.system('ffmpeg -y -loglevel error -i examples/{} -ar 16000 examples/tmp.wav'.format(ain))
        shutil.copyfile('examples/tmp.wav', 'examples/{}'.format(ain))

        # au embedding
        #from thirdparty.resemblyer_util.speaker_emb import get_spk_emb
        me, ae = get_spk_emb_prepr('examples/{}'.format(ain), resemblyzer_encoder)
        au_emb.append(me.reshape(-1))

        print('Processing audio file', ain)
        c = AutoVC_mel_Convertor('examples')

        au_data_i = c.convert_single_wav_to_autovc_input(audio_filename=os.path.join('examples', ain),
              autovc_model_path=opt_parser.load_AUTOVC_name)
        au_data += au_data_i
    if(os.path.isfile('examples/tmp.wav')):
        os.remove('examples/tmp.wav')

    # landmark fake placeholder
    fl_data = []
    rot_tran, rot_quat, anchor_t_shape = [], [], []
    for au, info in au_data:
        au_length = au.shape[0]
        fl = np.zeros(shape=(au_length, 68 * 3))
        fl_data.append((fl, info))
        rot_tran.append(np.zeros(shape=(au_length, 3, 4)))
        rot_quat.append(np.zeros(shape=(au_length, 4)))
        anchor_t_shape.append(np.zeros(shape=(au_length, 68 * 3)))

    if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl.pickle'))):
        os.remove(os.path.join('examples', 'dump', 'random_val_fl.pickle'))
    if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))):
        os.remove(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))
    if(os.path.exists(os.path.join('examples', 'dump', 'random_val_au.pickle'))):
        os.remove(os.path.join('examples', 'dump', 'random_val_au.pickle'))
    if (os.path.exists(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))):
        os.remove(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))

    with open(os.path.join('examples', 'dump', 'random_val_fl.pickle'), 'wb') as fp:
        pickle.dump(fl_data, fp)
    with open(os.path.join('examples', 'dump', 'random_val_au.pickle'), 'wb') as fp:
        pickle.dump(au_data, fp)
    with open(os.path.join('examples', 'dump', 'random_val_gaze.pickle'), 'wb') as fp:
        gaze = {'rot_trans':rot_tran, 'rot_quat':rot_quat, 'anchor_t_shape':anchor_t_shape}
        pickle.dump(gaze, fp)


    !pwd
    model = Audio2landmark_model(opt_parser, jpg_shape=shape_3d)
    if(len(opt_parser.reuse_train_emb_list) == 0):
        model.test(au_emb=au_emb)
    else:
        model.test(au_emb=None)


    fls = glob.glob1('examples', 'pred_fls_*.txt')
    fls.sort()

    for i in range(0,len(fls)):
        fl = np.loadtxt(os.path.join('examples', fls[i])).reshape((-1, 68,3))
        fl[:, :, 0:2] = -fl[:, :, 0:2]
        fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift

        if (ADD_NAIVE_EYE):
            fl = util.add_naive_eye(fl)

        # additional smooth
        fl = fl.reshape((-1, 204))
        fl[:, :48 * 3] = savgol_filter(fl[:, :48 * 3], 15, 3, axis=0)
        fl[:, 48*3:] = savgol_filter(fl[:, 48*3:], 5, 3, axis=0)
        fl = fl.reshape((-1, 68, 3))

        ''' STEP 6: Imag2image translation '''
        model = Image_translation_block(opt_parser, single_test=True)
        with torch.no_grad():
            model.single_test(jpg=img, fls=fl, filename=fls[i], prefix=opt_parser.jpg.split('.')[0])
            print('finish image2image gen')
        os.remove(os.path.join('examples', fls[i]))

        OUTPUT_MP4_NAME = '{}_pred_fls_{}_audio_embed.mp4'.format(
                                                                  opt_parser.jpg.split('.')[0],
                                                                  ain.split('.')[0]
                                                                  )


        %cd /content
        return "/content/MakeItTalk/examples/my_new_img_pred_fls_audio_audio_embed.mp4"#'/content/MakeItTalk/examples/{}'.format(OUTPUT_MP4_NAME)


<>:117: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:117: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-4-50602cd231cb>:117: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  ains = [item for item in ains if item is not 'tmp.wav']


In [ ]:
#@title Run
def setter(lip_x, lip_y, head_pose_motion, naive_eye, close_mouth, upper_lip,
           lower_lip, lip_WA, larger_eye, temperature, back, mem_len):

    global AMP_LIP_SHAPE_X
    AMP_LIP_SHAPE_X=lip_x
    global AMP_LIP_SHAPE_Y
    AMP_LIP_SHAPE_Y =lip_y
    global AMP_HEAD_POSE_MOTION
    AMP_HEAD_POSE_MOTION=head_pose_motion
    global ADD_NAIVE_EYE
    ADD_NAIVE_EYE=naive_eye
    global CLOSE_INPUT_FACE_MOUTH
    CLOSE_INPUT_FACE_MOUTH=close_mouth
    global UPPER_LIP_ADJUST
    UPPER_LIP_ADJUST=upper_lip
    global LOWER_LIP_ADJUST
    LOWER_LIP_ADJUST=lower_lip
    global LIP_WIDTH_ADJUST
    LIP_WIDTH_ADJUST=lip_WA
    global LARGER_EYE
    LARGER_EYE=larger_eye
    global TEMPERATURE
    TEMPERATURE=temperature
    global BACKGROUND
    BACKGROUND=back
    global MEMORY_LEN
    MEMORY_LEN =mem_len

typer=''
def upload_wav(file):
    #global voice_path
    if os.path.exists('/content/MakeItTalk/examples/my_own_voice_file.wav'):
        os.remove('/content/MakeItTalk/examples/my_own_voice_file.wav')
    with open('/content/MakeItTalk/examples/my_own_voice_file.wav', mode='bx') as f:
        f.write(file)

    #file_name=file.name.split('/')[-1]
    #shutil.move(file.name, '/content/MakeItTalk/examples/'+file_name)

def upload_img(img):
    # Load the image as a PIL object
    #image = Image.fromarray(img)
    global default_head_name
    #default_head_name=img.split('/')[-1].split('.')[0]
    fn='/content/MakeItTalk/examples/my_new_img.jpg'
    if os.path.exists(fn):
        os.remove(fn)
    #with open(fn, mode='bx') as f:
        #f.write(img)
    #image=image.save(fn)
    shutil.move(img,fn)
    default_head_name=fn.split('/')[-1].split('.')[0]

    preprocess()


def get_video(text):
    return generation(text)


with gr.Blocks() as demo:
    with gr.Tab(label = "Settings"):
        with gr.Row():
            with gr.Column():
                lip_x=gr.Slider(0.5, 5, 2, step=0.1, label="Lip_shape_X", interactive=True)
                lip_y=gr.Slider(0.5, 5, 2, step=0.1, label="Lip_shape_Y", interactive=True)

                naive_eye=gr.Checkbox(value=True, label="Add_naive_eye_blink", interactive=True)
                close_inp_mouth=gr.Checkbox(value=False, label="Close_input_mouth", interactive=True)

                head_pose_modtion=gr.Slider(0, 1, 0.35, step=0.05, label="Head_motion", interactive=True)
                upper_lip=gr.Slider(-1, 3, 1, step=1, label="Upper_lip", interactive=True)
                lower_lip=gr.Slider(-1, 3, 1, step=1, label="Lower_lip", interactive=True)
                lip_width=gr.Slider(0.8, 1.2, 1, step=0.01, label="Lip_width", interactive=True)
                larer_eye=gr.Slider(0, 4, 2, step=0.1, label="Larger_eye", interactive=True)
                temperature=gr.Slider(0, 1, 0.7, step=0.01, label="Temperature", interactive=True)
                mem_len=gr.Slider(1, 10, 2, step=1, label="Memory_len", interactive=True)

            with gr.Column():
                im_up=gr.inputs.Image(type="filepath")
                #im_up=gr.File(label="jpg", file_types=['image'], type="binary",interactive=True)
                im_up.upload(fn=upload_img, inputs=im_up)

                file_output = gr.File(label="wav", file_types=['audio'], type="binary",interactive=True)
                file_output.upload(upload_wav, inputs=file_output)
                #upload_button = gr.UploadButton("Upload a Audio", file_types=["audio"], file_count="single")
                #upload_button.upload(upload_wav, upload_button, file_output)

                background=gr.Text(lines=7)

        btn1 = gr.Button("Set")
        btn1.click(fn=setter, inputs=[lip_x, lip_y, head_pose_modtion, naive_eye, close_inp_mouth,
                                          upper_lip, lower_lip, lip_width, larer_eye, temperature, background,
                                          mem_len])

    with gr.Tab(label = "Generation"):
        show_result=gr.Video(type="file", label="result", autoplay=True, height=512) #interactive=True) #every=20,
        response=gr.Text(lines=1)
        btn2 = gr.Button("Generate")
        btn2.click(fn=get_video, inputs=response, outputs=show_result)

demo.launch(debug=True,  enable_queue=True, share=True)#debug=True) #share=True,

<ipython-input-5-91c9167358f4>:81: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  im_up=gr.inputs.Image(type="filepath")
<ipython-input-5-91c9167358f4>:81: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  im_up=gr.inputs.Image(type="filepath")
<ipython-input-5-91c9167358f4>:98: GradioUnusedKwargWarning: You have unused kwarg parameters in Video, please remove them: {'type': 'file'}
  show_result=gr.Video(type="file", label="result", autoplay=True, height=512) #interactive=True) #every=20,
<ipython-input-5-91c9167358f4>:103: GradioDeprecationWarning: The `enable_queue` parameter has been deprecated. Please use the `.queue()` method instead.
  demo.launch(debug=True,  enable_queue=True, share=True)#debug=True) #share=True,


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://824032a0a8e2090f48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://824032a0a8e2090f48.gradio.live


In [ ]:
2+2

4